In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

import helper

from selenium.common.exceptions import TimeoutException

import numpy as np

In [2]:
course_schedule = []
NO_WINDOW = True
IMPLICIT_WAIT_TIME = 60
EXPLICIT_WAIT_TIME = 60
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
# SCREEN_WIDTH = 1920
# SCREEN_HEIGHT = 1080
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
# SCREEN_WIDTH = 1920
# SCREEN_HEIGHT = 1200
# SCREEN_WIDTH = 3200
# SCREEN_HEIGHT = 1800
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

 # SETTING

In [3]:
YEAR = 2023
MONTH = 6
DAY = 17

schedule_filename = "19 June - 25 June 2023.json"

DELAY_BOOKING_TIMES = np.arange(0.5, 1.5, 0.2).tolist()
TIME_INTERVAL_SIGNIN_BOOKING = 600 #10 minutes

In [4]:
from tkinter import Y

day = DAY + 4

if MONTH in [1, 3, 5, 7, 8, 10, 12]:
    if day > 31:
        day = day % 31
elif MONTH in [4, 6, 9, 11]:
    if day > 30:
        day = day % 30
elif MONTH is "2":
    if YEAR % 4 != 0:
        if day > 28:
            day = day % 28
    else:
        if day > 29:
            day = day % 29

    
if day / 10 < 1:
    Day_string = "0{}".format(day)
else:
    Day_string = "{}".format(day)


print(Day_string)


21


In [5]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)

In [7]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)


In [8]:
def sign_in(driver, wait):
    
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')

    send_keys(username, "DN20092360")
    send_keys(password, "jj1216")
#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [9]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [10]:
with open(r'./class_schedules/{}'.format(schedule_filename), "r") as read_file:
    course_schedule = json.load(read_file)

### Place Test Code Block for Debugging here. The block is "stored" at the end of the notebook. Then commnet out the Production Code Block 

In [11]:
# ....

### Production Code Block

In [12]:
book_datetime = datetime.datetime(YEAR, MONTH, DAY, 22, 0, 0)

temp = []

# ##---------------------------------------------------
# ##

# # Monday
# #

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Gen', c)
# # c = helper.search_courses_by_teacher('Min', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Bar', c)
# # # c = helper.search_courses_by_teacher('', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)


# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Tone', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Hatha', c)
# # c = helper.search_courses_by_teacher('Tum', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Hatha', c)
# # c = helper.search_courses_by_teacher('Marc', c)
# # # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('核心', c)
# # c = helper.search_courses_by_teacher('El', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Tuesday
# #

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Gen', c)
# c = helper.search_courses_by_teacher('Tumai', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Wheel', c)
# c = helper.search_courses_by_teacher('Sim', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Gentle', c)
# # c = helper.search_courses_by_teacher('Mar', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Gentle', c)
# c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Vinyasa In', c)
# c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Abel', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

#  Wednesday 
#

c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Mys', c)
c = helper.search_courses_by_teacher('Si', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)


c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Wall', c)
# c = helper.search_courses_by_teacher('Char', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Basic', c)
c = helper.search_courses_by_teacher('Joe', c)
c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('War', c)
c = helper.search_courses_by_teacher('El', c)
c = helper.search_courses_by_time('PM', c)
temp.extend(c)
# 

# # Thursday
# #

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Mys', c)
# # c = helper.search_courses_by_teacher('Tumai', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)


# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Tone', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('', c)
# # c = helper.search_courses_by_teacher('Marc', c)
# # # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Mei', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Yin', c)
# c = helper.search_courses_by_teacher('Ell', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Friday
# #
# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Mysore I', c)
# # c = helper.search_courses_by_teacher('', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Yoga Stretch 伸展瑜珈', c)
# # c = helper.search_courses_by_teacher('Soni', c)
# # # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)


# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Heal', c)
# c = helper.search_courses_by_teacher('Tumai', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Tone', c)
# c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Whee', c)
# # c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Vin', c)
# # c = helper.search_courses_by_teacher('Marc', c)
# # # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Gen', c)
# # c = helper.search_courses_by_teacher('El', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Gen', c)
# # c = helper.search_courses_by_teacher('Char', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)
# #

# # Saturday
# #

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Ash', c) 
# # c = helper.search_courses_by_teacher('Sim', c)
# # # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# # c = helper.search_courses_by_name('Yin', c)
# c = helper.search_courses_by_teacher('Ja', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Yin', c)
# c = helper.search_courses_by_teacher('Moor', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Flow', c)
# c = helper.search_courses_by_teacher('Moor', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Sunday
# #

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Coco', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Gen', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # c = helper.search_courses_by_date(Day_string, course_schedule)
# # # c = helper.search_courses_by_name('Universal In', c)
# # c = helper.search_courses_by_teacher('Char', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Universal In', c)
# c = helper.search_courses_by_teacher('YaYa', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Yin', c)
# c = helper.search_courses_by_teacher('YaYa', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Aerial', c)
# c = helper.search_courses_by_teacher('Eunice', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

##---------------------------------------------------

courses_to_be_booked = temp

##---------------------------------------------------
##

# courses_to_be_booked = []
# courses_to_be_booked.append(temp[3])
# courses_to_be_booked.append(temp[0])
# courses_to_be_booked.append(temp[1])   
# courses_to_be_booked.append(temp[2])

##
##---------------------------------------------------

##---------------------------------------------------
##

# courses_to_be_booked = []

# course = {}
# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=747149'
# course['name'] = '艾楊格 壁繩'
# course['teacher'] = 'Yvette'
# course['date'] = '19 Oct Mon'
# course['time'] = '07:15 - 08:45 PM'
# courses_to_be_booked.append(course)

##
##---------------------------------------------------

helper.list_courses_to_be_booked(courses_to_be_booked)

# Set Sign In time 10 minutes before the open for booking time
signin_datetime = book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)

-----------------------
Date: 21 Jun Wed
-----------------------

Name: Mysore
Teacher: Simon Chang
Time: 06:30 - 09:00 AM

Name: Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩
Teacher: Charmian
Time: 02:30 - 03:30 PM

Name: Aerial Yoga Basic 空中瑜珈入門
Teacher: Joe
Time: 12:45 - 02:00 PM

Name: Aerial Yoga 空中瑜珈(低空)
Teacher: Joe
Time: 08:45 - 09:45 PM

Name: Gentle Yoga 和緩瑜珈
Teacher: Ellen
Time: 07:45 - 08:45 PM

Name: Warm Stretch 溫和伸展
Teacher: Ellen
Time: 06:30 - 07:30 PM
-----------------------


In [13]:
book_datetimes = [book_datetime + datetime.timedelta(seconds=DELAY_BOOKING_TIME) for DELAY_BOOKING_TIME in DELAY_BOOKING_TIMES]

In [14]:
def scheduled_sign_in(driver, wait):
#     print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [15]:
reserve_class_log_dicts = []

def reserve_class(driver, wait, course_to_be_booked):
    
    print("reserve_class: ({})\n".format(course_to_be_booked['time']))
    
    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    
    reserve_class_log_dict = {}
    
    reserve_class_log_dict['name'] = class_name
    
    now1 = datetime.datetime.now()
    driver.get(book_url)
    now2 = datetime.datetime.now()

    now3 = datetime.datetime.now()
    now4 = datetime.datetime.now()
    if click_book_this_class_now(driver, wait):
        now4 = datetime.datetime.now()
    else:
        now4 = datetime.datetime.now()

    reserve_class_log_dict['datetime1'] = now1
    reserve_class_log_dict['datetime2'] = now2
    reserve_class_log_dict['datetime3'] = now3
    reserve_class_log_dict['datetime4'] = now4
    
    reserve_class_log_dicts.append(reserve_class_log_dict)
    

In [16]:
def show_all_booking_result(reserve_class_log_dicts):
    
    print("Scheduling Logs:")
    
    for reserve_class_log_dict in reserve_class_log_dicts:

        print("Class: ", reserve_class_log_dict['name'])
        print("-----------------------------------------\n")
        print("\t** Begin: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime1']))
        print("\t** End: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime2']))

        print("\t**Begin: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime3']))
        print("\t**End: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime4']))
        print("-----------------------------------------\n")

In [17]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

idx_1 = 0

for book_datetime in book_datetimes:
    
    for idx_2,course_to_be_booked in enumerate(courses_to_be_booked):

        driver, wait = create_new_driver()
        driver.set_window_position(WINDOW_WIDTH * (idx_2 % WINDOWS_GRID_X), WINDOW_HEIGHT * int(idx_2 / WINDOWS_GRID_X));
        driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
        signin_datetime_var = signin_datetime + datetime.timedelta(seconds=idx_1*3)
        book_datetime_var = book_datetime + datetime.timedelta(seconds=idx_2*0.01)
        print("idx: ", idx_1, "scheduled signin datetime: ", signin_datetime_var)
        print("idx: ", idx_2, "scheduled book datetime: ", book_datetime_var)

        scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime_var, args=[driver, wait])
        scheduler.add_job(reserve_class, 'date', run_date= book_datetime_var, args=[driver, wait, course_to_be_booked])
    
        idx_1 = idx_1 + 1

scheduler.add_job(show_all_booking_result, 'date', run_date= book_datetime + datetime.timedelta(seconds=6), args=[reserve_class_log_dicts])
        
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()
 


[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


idx:  0 scheduled signin datetime:  2023-06-17 21:50:00
idx:  0 scheduled book datetime:  2023-06-17 22:00:00.500000
 


[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


idx:  1 scheduled signin datetime:  2023-06-17 21:50:03
idx:  1 scheduled book datetime:  2023-06-17 22:00:00.510000
 


...

idx:  26 scheduled signin datetime:  2023-06-17 21:51:18
idx:  2 scheduled book datetime:  2023-06-17 22:00:01.320000
 


[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


idx:  27 scheduled signin datetime:  2023-06-17 21:51:21
idx:  3 scheduled book datetime:  2023-06-17 22:00:01.330000
 


[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


idx:  28 scheduled signin datetime:  2023-06-17 21:51:24
idx:  4 scheduled book datetime:  2023-06-17 22:00:01.340000
 
idx:  29 scheduled signin datetime:  2023-06-17 21:51:27
idx:  5 scheduled book datetime:  2023-06-17 22:00:01.350000


In [18]:
###### print("https://www.trueclassbooking.com.tw/member/login.aspx\n")
for idx, course_to_be_booked in enumerate(courses_to_be_booked):
    print("Class: {} \n\turl: {}\n".format(course_to_be_booked['name'], course_to_be_booked['href']))

Class: Mysore 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1193948

Class: Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1193991

Class: Aerial Yoga Basic 空中瑜珈入門 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1193950

Class: Aerial Yoga 空中瑜珈(低空) 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1193954

Class: Gentle Yoga 和緩瑜珈 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1193993

Class: Warm Stretch 溫和伸展 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1194022

